In [3]:
# !git clone https://github.com/SZTankWang/ml-weibo-project-.git
%pip install paddlepaddle
%pip install jieba --upgrade
# -tiny==1.6.1

Note: you may need to restart the kernel to use updated packages.
Requirement already up-to-date: jieba in c:\programdata\anaconda3\lib\site-packages (0.42.1)Note: you may need to restart the kernel to use updated packages.



In [39]:
import jieba
import paddle
import re
import pandas as pd
import time
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor

import math as m
import random as r
import numpy as np

paddle.enable_static()
jieba.enable_paddle()# 启动paddle模式。 0.40版之后开始支持，早期版本不支持

Paddle enabled successfully......
2021-05-08 21:22:50,511 - DEBUG - Paddle enabled successfully......


In [4]:
df = pd.read_excel("data/raw_data.xlsx",skiprows=1,usecols=["微博正文","点赞数","转发数","评论数"])
df.head()

C:\ProgramData\Anaconda3\lib\site-packages\xlrd\xlsx.py:266: DeprecationWarning: This method will be removed in future versions.  Use 'tree.iter()' or 'list(tree.iter())' instead.
  for elem in self.tree.iter() if Element_has_iter else self.tree.getiterator():
C:\ProgramData\Anaconda3\lib\site-packages\xlrd\xlsx.py:312: DeprecationWarning: This method will be removed in future versions.  Use 'tree.iter()' or 'list(tree.iter())' instead.
  for elem in self.tree.iter() if Element_has_iter else self.tree.getiterator():


,微博正文,点赞数,转发数,评论数
0,【#你好，明天#】岁末年初，即便你的年终盘点一言难尽，但新年的flag仍然计日可期。需要抓住...,5389,1997,406
1,【今天，发条微博，向这些“闪亮的名字”致敬】南仁东、林俊德、张超、王继才、黄群、宋月才、姜开...,6826,1268,289
2,【夜读：这一年，谢谢自己】转眼间，2018年就要过去了。有时候我们很怕，怕自己的成长速度跟不...,6565,4326,580
3,【小调查：跨年夜，你如何度过的？】#2018最后一天#，你是如何度过的？陪在家人身边？宅在家...,919,2562,5281
4,【揭秘！习主席书架上的新变化】今晚，习近平主席发表了#2019新年贺词#。这一次，习主席书架...,6513,1758,440


In [5]:
df = df.rename(columns = {'微博正文':'text', '点赞数':'like', '转发数':'comment','评论数':'forward'}, inplace=False)
df.head()

,text,like,comment,forward
0,【#你好，明天#】岁末年初，即便你的年终盘点一言难尽，但新年的flag仍然计日可期。需要抓住...,5389,1997,406
1,【今天，发条微博，向这些“闪亮的名字”致敬】南仁东、林俊德、张超、王继才、黄群、宋月才、姜开...,6826,1268,289
2,【夜读：这一年，谢谢自己】转眼间，2018年就要过去了。有时候我们很怕，怕自己的成长速度跟不...,6565,4326,580
3,【小调查：跨年夜，你如何度过的？】#2018最后一天#，你是如何度过的？陪在家人身边？宅在家...,919,2562,5281
4,【揭秘！习主席书架上的新变化】今晚，习近平主席发表了#2019新年贺词#。这一次，习主席书架...,6513,1758,440


In [ ]:
stopwords = []
with open('data/stopwords.txt') as file:
    for i in file:
        stopwords.append(i.strip('\n'))

cutwordslist = []
oplis = []
sentences = df.text.to_list()
f = open("data/tfidf.txt", "w")

for line in sentences[]:
    linelis = list(line)
    newlis = []
    for i in linelis:
        if re.match('[\w]', i):
            newlis.append(i)
        else:
            newlis.append('')
    newline = ''.join(newlis)
    
    # cutwordslist.append([word for word in jieba.cut(newline, use_paddle=True) if word not in stopwords])

    line = ' '.join([word for word in jieba.cut(newline, use_paddle=True) if word not in stopwords])
    f.write(line)
    f.write('\n')
    # oplis.append(line)
    # afterTfIdf.append()
# print(oplis)
# f = open("/content/ml-weibo-project-/tfidf.txt", "w")
# f.write(line)
f.close()


In [40]:
f = open("data/tfidf.txt", 'r',encoding="utf8")
allsentences = f.readlines()

for i in range(len(allsentences)):
  allsentences[i] = allsentences[i].strip('\n')
y = df[['like','comment','forward']]

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(allsentences, y, train_size=.7, test_size=.3, random_state=0, shuffle=True, stratify=None)
f.close()

In [42]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

 
#该类会将文本中的词语转换为词频矩阵，矩阵元素a[i][j] 表示j词在i类文本下的词频
vectorizer = CountVectorizer()
#该类会统计每个词语的tf-idf权值
tf_idf_transformer = TfidfTransformer()
#将文本转为词频矩阵并计算tf-idf
tf_idf = tf_idf_transformer.fit_transform(vectorizer.fit_transform(X_train))
#将tf-idf矩阵抽取出来，元素a[i][j]表示j词在i类文本中的tf-idf权重
# X_train_weight = np.asarray(tf_idf)
# print(X_train_weight)
# X_train_weight = tf_idf.toarray().astype(float)
 
print(type(tf_idf))    


# X_test_weight = tf_idf.toarray().astype(float)  # 测试集TF-IDF权重矩阵
# X_test_weight.shape 
#print('输出x_train文本向量：')
#print(X_train_weight)
#print('输出x_test文本向量：')
#print(X_test_weight)

print(type(y_train))
# model = LinearRegression().fit(X_train_weight, y_train)
# y_train_pred = model.predict(X_train_weight)
# y_test_pred = model.predict(X_test_weight)

# train_error = sklearn.metrics.mean_squared_error(y_train, y_train_pred)
# test_error = sklearn.metrics.mean_squared_error(y_test, y_test_pred)


# use SGD partial fit
model = SGDRegressor(loss='squared_loss')

count = 0
start = 0
end = 0
while count < (len(X_test)//10000):
    start = count * 10000 
    end = min((count+1) * 10000,len(X_test)-1)
    count += 1
    
    model.partial_fit(tf_idf[start:end].toarray(),y_train[start:end])

#对测试集进行tf-idf权重计算
# tf_idf = tf_idf_transformer.transform(vectorizer.transform(X_test))

<class 'scipy.sparse.csr.csr_matrix'>
<class 'pandas.core.frame.DataFrame'>


ValueError: y should be a 1d array, got an array of shape (10000, 3) instead.

In [37]:
len(X_train) // 10000

7